In [4]:
import requests
from bs4 import BeautifulSoup
import html5lib
#proxies = {'http': 'http://proxy.charite.de:8080',

#           'https': 'https://proxy.charite.de:8080'}
#print("Using HTTP proxy %s" % proxies['http'])

In [5]:
curURL = 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Spezial:MultiCategorySearch&limit=500&offset=0&wpInCategory1=Gl%C3%BCcksspielsucht'
s      = requests.session()
#r      = s.get(curURL,proxies=proxies)
r      = s.get(curURL)
soup   = BeautifulSoup(r.text,"html5lib")
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [7]:
from IPython.display import Image
url = 'http://www.openbookproject.net/tutorials/getdown/css/images/lesson4/HTMLDOMTree.png'
Image(url = url)

In [8]:
Image(url = 'http://www.cs.toronto.edu/~shiva/cscb07/img/dom/treeStructure.png')

In [9]:
links = soup.find_all("li")
print (type(links))
print(len(links))

<class 'bs4.element.ResultSet'>
142


In [10]:
print(links[0])

<li><a href="/shb/index.php?title=AG_Spielsucht" title="AG Spielsucht">AG Spielsucht</a></li>


In [11]:
contacts = {}
for element in links:
    contacts[element.a.get_text()] = {}
first_keys = list(contacts.keys())[0:5]
for key in first_keys:
    print(key)
    print(contacts[key])

Psychotherapeutische Praxis - Manouchehr Djafari
{}
Psychotherapeutische Praxis - Uwe Schneider
{}
FAIRPLAY des KKH e.V.
{}
Psychotherapeutische Praxis - Christoph Kussmaul
{}
Psychotherapeutische Praxis - Verena Engels
{}


In [12]:
print(links[0].a["href"])
prefix = 'http://www.spielsucht-hilfe-berlin.de'

/shb/index.php?title=AG_Spielsucht


In [13]:
for element in links:
    contacts[element.a.get_text()]["link"] = prefix + element.a["href"]

In [14]:
for key in first_keys:
    print(key)
    print(contacts[key])

Psychotherapeutische Praxis - Manouchehr Djafari
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Manouchehr_Djafari'}
Psychotherapeutische Praxis - Uwe Schneider
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Uwe_Schneider'}
FAIRPLAY des KKH e.V.
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=FAIRPLAY_des_KKH_e.V.'}
Psychotherapeutische Praxis - Christoph Kussmaul
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Christoph_Kussmaul'}
Psychotherapeutische Praxis - Verena Engels
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Verena_Engels'}


In [16]:
# now go the link and apply a function that reads out all the information
# do it with first one first
key = list(contacts.keys())[0:2][1]

#for key in first_keys:
print(key)
print(contacts[key])

# get the link
curLink = contacts[key]['link']
print(curLink)

s      = requests.session()
#r      = s.get(curLink,proxies=proxies)
r      = s.get(curLink)
ssoup  = BeautifulSoup(r.text,"html5lib")
print(type(ssoup))

Psychotherapeutische Praxis - Uwe Schneider
{'link': 'http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Uwe_Schneider'}
http://www.spielsucht-hilfe-berlin.de/shb/index.php?title=Psychotherapeutische_Praxis_-_Uwe_Schneider
<class 'bs4.BeautifulSoup'>


In [17]:
# functions
def getLink(linkEntry):
    return(linkEntry['link'])

def getInfos(link):
    """Takes the link of a help institution and extracts
    name, address, contact"""
    s = requests.session()
    #r = s.get(curLink,proxies=proxies)
    r = s.get(link)
    ssoup = BeautifulSoup(r.text,"html5lib")
    infos = ssoup.find_all("td") # td means line of a table
    try:
        retInfos = [infos[0].get_text(),infos[3].get_text(),infos[5].get_text()]
    except:
        print('infos not in expected format')
        retInfos = ['NA','NA','NA']
    return(retInfos)

In [21]:
ID = []
name = []
address = []
contact = []

for counter, value in enumerate(contacts):
    print (counter, end="\r")
    curLink = getLink(contacts[value])
    retInfos = getInfos(curLink)
    ID.append(value)
    name.append(retInfos[0])
    address.append(retInfos[1])
    contact.append(retInfos[2])

infos not in expected format
infos not in expected format
infos not in expected format
infos not in expected format
infos not in expected format
infos not in expected format
infos not in expected format
infos not in expected format


In [22]:
import pandas as pd
import openpyxl

#pd.DataFrame(ID)
#pd.DataFrame(name)

df = pd.DataFrame({'ID': ID, 'Name': name, 'Address': address,
             'Contact' : contact})

df.to_excel('Spielsucht_Hilfe.xlsx')

In [23]:
from openpyxl import load_workbook
from openpyxl.styles import Alignment

# multi-line wrap
wb = load_workbook('Spielsucht_Hilfe.xlsx')
ws = wb.active # wb.active returns a Worksheet object

for row in ws.iter_rows():
    for cell in row:
        cell.alignment = Alignment(wrapText=True)
    
    from openpyxl import Workbook

wb.save("Spielsucht_Hilfe_wrapped.xlsx")

#worksheet.cell('A1').style.alignment.wrap_text = True
#worksheet.cell('A1').value = "Line 1\nLine 2\nLine 3"

#workbook.save('wrap_text1.xlsx')
